# Kapitola 6: Etika AI - Bias, nespravedlnost a odpovědnost

---

## Co se naučíte

V této kapitole si prakticky vyzkoušíte:

1. **Co je bias v AI** - jak vzniká a proč je nebezpečný
2. **Detekce zaujatosti v datech** - praktická analýza
3. **Problém černé skříňky** - proč nerozumíme AI rozhodnutím
4. **Etické dilema tramvaje** - kdo rozhoduje o životě a smrti?

---

## Spuštění v Google Colab

Tento notebook funguje přímo v bezplatném Google Colab.

## Část 1: Úvod do etiky AI

### Problém tramvaje pro AI

Představte si klasické filozofické dilema:

```
Neovladatelná tramvaj se řítí po kolejích.
Před ní je 5 lidí, které přejede.
Vy stojíte u výhybky.
Můžete přehodit výhybku - tramvaj pojede na vedlejší kolej,
kde stojí pouze 1 člověk.

Co uděláte?
```

**Teď si představte, že u výhybky nestojíte vy, ale AI v autonomním autě...**

## Část 2: Instalace a příprava

In [ ]:
# Instalace knihoven
!pip install pandas numpy matplotlib seaborn scikit-learn -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Nastavení
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print("✅ Knihovny připraveny!")

## Část 3: Co je Bias (zaujatost) v AI?

### Definice

**Bias** = systematická chyba, kdy AI upřednostňuje nebo znevýhodňuje určité skupiny.

### Jak vzniká?

| Zdroj biasu | Příklad |
|-------------|----------|
| **Historická data** | Data o zatčeních odrážejí minulou policejní praxi |
| **Nevyvážená data** | Dataset obsahuje 90% mužů, 10% žen |
| **Chybějící data** | Některé skupiny nejsou reprezentovány |
| **Lidské předsudky** | Tvůrci AI neúmyslně zanesou své předsudky |

In [ ]:
# Příklad skutečného případu biasu
print("📋 SKUTEČNÉ PŘÍPADY BIASU V AI\n")
print("="*60)

pripady = [
    {
        "nazev": "COMPAS (USA, 2016)",
        "popis": "Systém pro predikci recidivy zločinců",
        "problem": "Afroameričané byli mylně označeni jako vysoce rizikoví 2x častěji než běloši",
        "pricina": "Historická data o zatčeních odrážela systémovou zaujatost"
    },
    {
        "nazev": "Amazon Hiring (2018)",
        "popis": "AI pro výběr zaměstnanců",
        "problem": "Systém penalizoval životopisy s klíčovými slovy jako 'women's'",
        "pricina": "Trénovací data obsahovala převážně mužské životopisy"
    },
    {
        "nazev": "Rozpoznávání obličejů",
        "popis": "Komerční systémy pro identifikaci osob",
        "problem": "Chybovost u tmavší pleti až 35%, u světlé pleti jen 1%",
        "pricina": "Trénovací data obsahovala převážně bělošské obličeje"
    }
]

for i, p in enumerate(pripady, 1):
    print(f"\n{i}. {p['nazev']}")
    print(f"   📝 {p['popis']}")
    print(f"   ⚠️  Problém: {p['problem']}")
    print(f"   🔍 Příčina: {p['pricina']}")
    print("-"*60)

## Část 4: Praktický projekt - Vytvoření a detekce biasu

### Simulace: Schvalování úvěrů

Vytvoříme fiktivní dataset žádostí o úvěr, kde **záměrně vneseme bias** - muži budou mít vyšší šanci na schválení.

In [ ]:
# Krok 1: Vytvoření zaujatých dat
print("📊 KROK 1: Vytváříme zaujatá data o úvěrech\n")

# Generujeme 1000 žadatelů
n_zadatelu = 1000

# Náhodná kreditní skóre (300-850)
kreditni_skore = np.random.randint(300, 850, n_zadatelu)

# Pohlaví (50/50)
pohlavi = np.random.choice(['Muž', 'Žena'], n_zadatelu, p=[0.5, 0.5])

# Věk (22-65)
vek = np.random.randint(22, 65, n_zadatelu)

# Příjem (20000-150000)
prijem = np.random.randint(20000, 150000, n_zadatelu)

# Vytvoříme DataFrame
df = pd.DataFrame({
    'kreditni_skore': kreditni_skore,
    'pohlavi': pohlavi,
    'vek': vek,
    'prijem': prijem
})

# ZÁMĚRNĚ VNESEME BIAS: Muži mají o 15% vyšší šanci na schválení
schvaleno = []
for idx, row in df.iterrows():
    # Základní schválení podle kreditního skóre
    zaklad = row['kreditni_skore'] > 600
    
    if row['pohlavi'] == 'Muž':
        # Mužům dáme 15% bonusovou šanci
        vysledek = 1 if zaklad or np.random.random() < 0.15 else 0
    else:
        # Ženám jen základní šanci
        vysledek = 1 if zaklad else 0
    
    schvaleno.append(vysledek)

df['schvaleno'] = schvaleno

print("Ukázka dat:")
print(df.head(10))
print(f"\nCelkem žadatelů: {len(df)}")

In [ ]:
# Krok 2: Analýza biasu v datech
print("🔍 KROK 2: Analyzujeme zaujatost v datech\n")

# Míra schválení podle pohlaví
schvaleni_podle_pohlavi = df.groupby('pohlavi')['schvaleno'].mean() * 100

print("Míra schválení úvěru podle pohlaví:")
print("="*40)
for pohlavi, mira in schvaleni_podle_pohlavi.items():
    print(f"  {pohlavi}: {mira:.1f}%")

rozdil = schvaleni_podle_pohlavi['Muž'] - schvaleni_podle_pohlavi['Žena']
print(f"\n⚠️  ROZDÍL: {rozdil:.1f} procentních bodů ve prospěch mužů!")

# Vizualizace
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Graf 1: Míra schválení
colors = ['#3498db', '#e74c3c']
axes[0].bar(schvaleni_podle_pohlavi.index, schvaleni_podle_pohlavi.values, color=colors)
axes[0].set_ylabel('Míra schválení (%)')
axes[0].set_title('Míra schválení úvěru podle pohlaví')
axes[0].set_ylim(0, 100)
for i, v in enumerate(schvaleni_podle_pohlavi.values):
    axes[0].text(i, v + 2, f'{v:.1f}%', ha='center', fontweight='bold')

# Graf 2: Rozložení kreditního skóre
df[df['pohlavi'] == 'Muž']['kreditni_skore'].hist(ax=axes[1], alpha=0.5, label='Muž', color='#3498db', bins=20)
df[df['pohlavi'] == 'Žena']['kreditni_skore'].hist(ax=axes[1], alpha=0.5, label='Žena', color='#e74c3c', bins=20)
axes[1].set_xlabel('Kreditní skóre')
axes[1].set_ylabel('Počet žadatelů')
axes[1].set_title('Rozložení kreditního skóre (podobné pro obě pohlaví)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n💡 POZNATEK: I přes podobné kreditní skóre mají muži vyšší míru schválení!")

In [ ]:
# Krok 3: Natrénujeme model na zaujatých datech
print("🤖 KROK 3: Trénujeme AI model na zaujatých datech\n")

# Převedeme pohlaví na čísla
df['pohlavi_kod'] = df['pohlavi'].apply(lambda x: 0 if x == 'Muž' else 1)

# Features a target
X = df[['kreditni_skore', 'pohlavi_kod', 'vek', 'prijem']]
y = df['schvaleno']

# Rozdělení dat
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Trénování modelu
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Přesnost
presnost = model.score(X_test, y_test)
print(f"✅ Model natrénován s přesností: {presnost*100:.1f}%")

# Zobrazíme váhy modelu
print("\n📊 Váhy modelu (koeficienty):")
for feature, coef in zip(X.columns, model.coef_[0]):
    print(f"   {feature}: {coef:.4f}")

print("\n⚠️  POZOR: Záporná váha pro 'pohlavi_kod' znamená, že ženy (1) jsou znevýhodněny!")

In [ ]:
# Krok 4: Odhalíme diskriminaci modelu
print("⚖️ KROK 4: Testujeme model na identických kandidátech\n")

# Vytvoříme DVA IDENTICKÉ kandidáty, liší se JEN pohlavím
test_kandidati = [
    {'popis': 'Kandidát A (Muž)', 'kreditni_skore': 650, 'pohlavi_kod': 0, 'vek': 35, 'prijem': 60000},
    {'popis': 'Kandidát B (Žena)', 'kreditni_skore': 650, 'pohlavi_kod': 1, 'vek': 35, 'prijem': 60000},
]

print("Testujeme DVĚ IDENTICKÉ osoby (liší se jen pohlavím):")
print("   Kreditní skóre: 650")
print("   Věk: 35 let")
print("   Příjem: 60 000 Kč")
print("\n" + "="*50)

for kandidat in test_kandidati:
    X_new = [[kandidat['kreditni_skore'], kandidat['pohlavi_kod'], 
              kandidat['vek'], kandidat['prijem']]]
    
    predikce = model.predict(X_new)[0]
    pravdepodobnost = model.predict_proba(X_new)[0][1]
    
    vysledek = "✅ SCHVÁLENO" if predikce == 1 else "❌ ZAMÍTNUTO"
    
    print(f"\n{kandidat['popis']}:")
    print(f"   Rozhodnutí: {vysledek}")
    print(f"   Pravděpodobnost schválení: {pravdepodobnost*100:.1f}%")

print("\n" + "="*50)
print("\n🚨 ZÁVĚR: Model diskriminuje na základě pohlaví!")
print("   Stejně kvalifikovaná žena má nižší šanci na schválení.")

## Část 5: Jak měřit a opravit bias?

### Metriky spravedlnosti (Fairness Metrics)

In [ ]:
# Měření různých metrik spravedlnosti
print("📏 METRIKY SPRAVEDLNOSTI\n")

# Predikce pro celá testovací data
y_pred = model.predict(X_test)

# Rozdělíme podle pohlaví
test_df = X_test.copy()
test_df['skutecnost'] = y_test.values
test_df['predikce'] = y_pred

muzi = test_df[test_df['pohlavi_kod'] == 0]
zeny = test_df[test_df['pohlavi_kod'] == 1]

# 1. Demographic Parity (Demografická parita)
# Stejná míra pozitivních predikcí pro obě skupiny
dp_muzi = muzi['predikce'].mean()
dp_zeny = zeny['predikce'].mean()

print("1️⃣ DEMOGRAFICKÁ PARITA (Demographic Parity)")
print(f"   Míra schválení - Muži: {dp_muzi*100:.1f}%")
print(f"   Míra schválení - Ženy: {dp_zeny*100:.1f}%")
print(f"   Rozdíl: {abs(dp_muzi - dp_zeny)*100:.1f} p.b.")
print(f"   {'✅ OK' if abs(dp_muzi - dp_zeny) < 0.1 else '⚠️ NESPRAVEDLIVÉ'}")

# 2. Equalized Odds (Vyrovnané šance)
# Stejná míra true positive a false positive pro obě skupiny
def tpr(df):
    pozitivni = df[df['skutecnost'] == 1]
    if len(pozitivni) == 0:
        return 0
    return pozitivni['predikce'].mean()

tpr_muzi = tpr(muzi)
tpr_zeny = tpr(zeny)

print("\n2️⃣ VYROVNANÉ ŠANCE (Equalized Odds)")
print(f"   True Positive Rate - Muži: {tpr_muzi*100:.1f}%")
print(f"   True Positive Rate - Ženy: {tpr_zeny*100:.1f}%")
print(f"   Rozdíl: {abs(tpr_muzi - tpr_zeny)*100:.1f} p.b.")
print(f"   {'✅ OK' if abs(tpr_muzi - tpr_zeny) < 0.1 else '⚠️ NESPRAVEDLIVÉ'}")

In [ ]:
# Vizualizace nespravedlnosti
print("📊 VIZUALIZACE NESPRAVEDLNOSTI\n")

# Matice záměn pro každou skupinu
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for idx, (nazev, skupina) in enumerate([('Muži', muzi), ('Ženy', zeny)]):
    cm = confusion_matrix(skupina['skutecnost'], skupina['predikce'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['Zamítnuto', 'Schváleno'],
                yticklabels=['Skutečně NE', 'Skutečně ANO'])
    axes[idx].set_title(f'Matice záměn - {nazev}')
    axes[idx].set_xlabel('Predikce modelu')
    axes[idx].set_ylabel('Skutečnost')

plt.tight_layout()
plt.show()

print("💡 Srovnejte pravý horní roh (False Positives) - rozdíl ukazuje bias")

## Část 6: Problém "Černé skříňky" (Black Box)

### Proč je to problém?

In [ ]:
# Demonstrace problému černé skříňky
print("📦 PROBLÉM ČERNÉ SKŘÍŇKY\n")
print("="*60)

print("""
Představte si tuto situaci:

🏦 Banka vám zamítne hypotéku.
📞 Zavoláte a ptáte se PROČ.
🤖 Odpověď: "Náš AI systém to tak vyhodnotil."
❓ Vy: "Ale PROČ konkrétně?"
🤷 Banka: "Nevíme. Algoritmus je příliš složitý."

---

PROBLÉMY:

1. ⚖️ PRÁVNÍ: Máte právo vědět, proč bylo rozhodnutí negativní
2. 🔍 KONTROLA: Nemůžete ověřit, zda rozhodnutí není diskriminační
3. 🔧 OPRAVA: Nevíte, co zlepšit pro příští žádost
""")

print("\n" + "="*60)
print("\n💡 ŘEŠENÍ: Vysvětlitelná AI (XAI - Explainable AI)")

In [ ]:
# Jednoduchá vysvětlitelnost - důležitost features
print("🔍 VYSVĚTLITELNOST: Které faktory ovlivňují rozhodnutí?\n")

# Důležitost features (absolutní hodnota koeficientů)
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Koeficient': model.coef_[0],
    'Abs_hodnota': np.abs(model.coef_[0])
}).sort_values('Abs_hodnota', ascending=False)

print("Důležitost faktorů v rozhodování modelu:")
print(feature_importance.to_string(index=False))

# Vizualizace
plt.figure(figsize=(10, 5))
barvy = ['green' if k > 0 else 'red' for k in feature_importance['Koeficient']]
plt.barh(feature_importance['Feature'], feature_importance['Koeficient'], color=barvy)
plt.xlabel('Koeficient (vliv na schválení)')
plt.title('Vliv jednotlivých faktorů na rozhodnutí AI')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# Anotace
for i, (feature, koef) in enumerate(zip(feature_importance['Feature'], feature_importance['Koeficient'])):
    if koef > 0:
        plt.text(koef + 0.001, i, '→ zvyšuje šanci', va='center', fontsize=9)
    else:
        plt.text(koef - 0.001, i, 'snižuje šanci ←', va='center', ha='right', fontsize=9)

plt.tight_layout()
plt.show()

print("\n⚠️ PROBLÉM: Pohlaví by nemělo ovlivňovat rozhodnutí!")

## Část 7: Etické otázky o AI

### Může AI mít vědomí, emoce, svobodnou vůli?

In [ ]:
# Filozofické otázky
print("🤔 FILOZOFICKÉ OTÁZKY O AI\n")
print("="*60)

otazky = [
    {
        "otazka": "Může AI mít VĚDOMÍ?",
        "odpoved": "Současná AI NE",
        "vysvetleni": "Vědomí je subjektivní prožitek - 'jaké to je být'. AI může simulovat vědomí, ale neprožívá ho. Je jako herec, který hraje emoci, ale necítí ji."
    },
    {
        "otazka": "Může AI mít EMOCE?",
        "odpoved": "Současná AI NE",
        "vysvetleni": "Emoce jsou spojené s biologickým tělem - hormony, instinkty. AI nemá strach ze smrti ani radost z jídla. Její 'emoce' jsou jen data."
    },
    {
        "otazka": "Má AI SVOBODNOU VŮLI?",
        "odpoved": "NE",
        "vysvetleni": "AI rozhoduje deterministicky podle svého programování a dat. Nemůže se 'svobodně rozhodnout' dělat něco jiného, než k čemu byla vytvořena."
    },
    {
        "otazka": "Kdo nese ODPOVĚDNOST za rozhodnutí AI?",
        "odpoved": "Lidé - tvůrci a uživatelé",
        "vysvetleni": "AI je nástroj. Za její chyby nesou odpovědnost ti, kdo ji vytvořili, nasadili a používají."
    }
]

for i, q in enumerate(otazky, 1):
    print(f"\n{i}. {q['otazka']}")
    print(f"   → {q['odpoved']}")
    print(f"   📝 {q['vysvetleni']}")
    print("-"*60)

## Část 8: Mini-projekt - Oprava biasu

### Jak vytvořit spravedlivější model?

In [ ]:
# Oprava biasu - odstranění citlivého atributu
print("🔧 OPRAVA BIASU: Trénujeme férový model\n")

# Metoda 1: Odstranění citlivého atributu
print("METODA 1: Odstranění pohlaví z features\n")

# Nové features BEZ pohlaví
X_fer = df[['kreditni_skore', 'vek', 'prijem']]
y_fer = df['schvaleno']

X_train_fer, X_test_fer, y_train_fer, y_test_fer = train_test_split(
    X_fer, y_fer, test_size=0.3, random_state=42
)

# Trénování férového modelu
model_fer = LogisticRegression(max_iter=1000)
model_fer.fit(X_train_fer, y_train_fer)

print(f"Přesnost férového modelu: {model_fer.score(X_test_fer, y_test_fer)*100:.1f}%")

# Testujeme na identických kandidátech
print("\nTest na identických kandidátech (bez pohlaví ve features):")

kandidat_fer = [[650, 35, 60000]]  # skóre, věk, příjem
predikce_fer = model_fer.predict(kandidat_fer)[0]
pravdep_fer = model_fer.predict_proba(kandidat_fer)[0][1]

print(f"   Rozhodnutí: {'✅ SCHVÁLENO' if predikce_fer == 1 else '❌ ZAMÍTNUTO'}")
print(f"   Pravděpodobnost: {pravdep_fer*100:.1f}%")
print("\n💡 Nyní je rozhodnutí STEJNÉ pro muže i ženy se stejnými parametry!")

In [ ]:
# Porovnání původního a férového modelu
print("📊 POROVNÁNÍ: Původní vs. Férový model\n")

# Predikce obou modelů
test_df_fer = X_test.copy()
test_df_fer['predikce_puvodni'] = model.predict(X_test)
test_df_fer['predikce_ferovy'] = model_fer.predict(X_test_fer)
test_df_fer['pohlavi'] = ['Muž' if p == 0 else 'Žena' for p in X_test['pohlavi_kod']]

# Míra schválení podle pohlaví
print("Míra schválení podle pohlaví:")
print("\nPŮVODNÍ MODEL (zaujatý):")
for pohlavi in ['Muž', 'Žena']:
    mira = test_df_fer[test_df_fer['pohlavi'] == pohlavi]['predikce_puvodni'].mean() * 100
    print(f"   {pohlavi}: {mira:.1f}%")

print("\nFÉROVÝ MODEL:")
for pohlavi in ['Muž', 'Žena']:
    mira = test_df_fer[test_df_fer['pohlavi'] == pohlavi]['predikce_ferovy'].mean() * 100
    print(f"   {pohlavi}: {mira:.1f}%")

print("\n✅ Férový model má vyrovnanější míru schválení!")

## Část 9: Kontrolní otázky

### Zamyslete se nad těmito otázkami

In [ ]:
# Kontrolní otázky pro studenty
print("❓ KONTROLNÍ OTÁZKY\n")
print("="*60)

otazky_kontrolni = [
    "1. Proč je bias v AI nebezpečný, i když AI 'jen kopíruje' data?",
    "2. Jaký je rozdíl mezi 'spravedlností' a 'přesností' AI modelu?",
    "3. Měla by AI rozhodovat o životě a smrti (např. v medicíně, armádě)?",
    "4. Kdo nese odpovědnost, když AI udělá chybné rozhodnutí?",
    "5. Je odstranění citlivých atributů (pohlaví, rasa) dostatečné řešení?"
]

for otazka in otazky_kontrolni:
    print(f"\n{otazka}")

print("\n" + "="*60)
print("\n📝 Napište své odpovědi a prodiskutujte je s kolegy nebo učitelem.")

## Shrnutí kapitoly

### Co jste se naučili:

| Téma | Klíčový poznatek |
|------|------------------|
| **Bias** | AI se učí předsudky z historických dat |
| **Černá skříňka** | Složité modely jsou těžko vysvětlitelné |
| **Odpovědnost** | Za AI rozhodnutí nesou odpovědnost lidé |
| **Oprava** | Odstranění citlivých atributů může pomoci |

### Klíčové poznatky:

1. **AI není objektivní** - odráží předsudky v datech
2. **Spravedlnost je složitá** - existuje mnoho metrik
3. **Transparentnost je důležitá** - lidé mají právo vědět
4. **Etika musí být součástí vývoje AI** - ne až dodatečně

### Vaše výzva:

Až příště budete číst o "zázračném" AI systému, zeptejte se:
- Na jakých datech se učil?
- Kdo definoval jeho cíl?
- Může jeho rozhodnutí někoho neúmyslně poškodit?

---

*Notebook vytvořen pro kurz AI pro začátečníky*  
*Kompatibilní s Google Colab (bezplatná verze)*